# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [36]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

from collections import Counter
from nltk import ngrams

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Is\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
# Read your api key environment variable
load_dotenv()
api_key= os.getenv("NEWS_API_KEY")

In [5]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [9]:
# Fetch the Bitcoin news articles
bitcoin = newsapi.get_everything(q='bitcoin',
                                      language='en',
                                      sort_by='relevancy'
                                      )
print(f"Total articles about Bitcoin: {bitcoin['totalResults']}")

Total articles about Bitcoin: 8493


In [10]:
# Fetch the Ethereum news articles
ethereum = newsapi.get_everything(q='Ethereum', language="en", sort_by="relevancy")
print(f"Total articles about Ethereum: {ethereum['totalResults']}")

Total articles about Ethereum: 2924


In [13]:
# Create the Bitcoin sentiment scores DataFrame
bitcoin_sentiments = []
for article in bitcoin["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        # append each element of each iteration to the btc_sentiments empty list
        
        bitcoin_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
        })
    except AttributeError:
        pass
    
bitcoin_df = pd.DataFrame(bitcoin_sentiments)
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
bitcoin_df = bitcoin_df[cols]

bitcoin_df.head()

,date,text,compound,positive,negative,neutral
0,2021-11-05,A similar hoax earlier this year tied Walmart ...,-0.2732,0.000,0.063,0.937
1,2021-10-10,"Specifically, why did someone make a massive p...",0.5461,0.121,0.000,0.879
2,2021-10-28,Theres a big new presence slurping up power fr...,0.3612,0.096,0.000,0.904
3,2021-10-26,"For all the talk of democratizing finance, the...",0.0000,0.000,0.000,1.000
4,2021-10-21,Representation of cryptocurrency Bitcoin is pl...,0.0000,0.000,0.000,1.000


In [16]:
# Create the Ethereum sentiment scores DataFrame
ethereum_sentiments = []
for article in ethereum["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        # append each element of each iteration to the ethereum_sentiments empty list
        ethereum_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
        })
    except AttributeError:
        pass
    
ethereum_df = pd.DataFrame(ethereum_sentiments)
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
ethereum_df = ethereum_df[cols]

ethereum_df.head()

,date,text,compound,positive,negative,neutral
0,2021-10-16,A new cross-chain bridge is currently connecte...,0.0000,0.000,0.0,1.000
1,2021-10-14,Mark Cuban has some advice for people who are ...,0.0000,0.000,0.0,1.000
2,2021-11-05,Ethereum and bitcoin are the two biggest crypt...,0.4588,0.094,0.0,0.906
3,2021-11-01,Elon Musk\r\npicture alliance / Getty Images\r...,0.5267,0.093,0.0,0.907
4,2021-11-01,Cryptocurrency and business continuity line im...,0.4588,0.097,0.0,0.903


In [17]:
# Describe the Bitcoin Sentiment
bitcoin_df.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.000000,20.00000
mean,0.195090,0.048400,0.003150,0.94845
std,0.288628,0.064743,0.014087,0.06379
min,-0.273200,0.000000,0.000000,0.80100
25%,0.000000,0.000000,0.000000,0.89650
50%,0.000000,0.000000,0.000000,1.00000
75%,0.481000,0.103500,0.000000,1.00000
max,0.755800,0.199000,0.063000,1.00000


In [18]:
# Describe the Ethereum Sentiment
ethereum_df.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.000000,20.000000
mean,0.285405,0.062800,0.002400,0.934800
std,0.313684,0.074233,0.010733,0.077756
min,0.000000,0.000000,0.000000,0.779000
25%,0.000000,0.000000,0.000000,0.902500
50%,0.180600,0.036000,0.000000,0.964000
75%,0.530600,0.097500,0.000000,1.000000
max,0.822500,0.221000,0.048000,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: Ethereum had the highest mean at 0.062800

Q: Which coin had the highest compound score?

A: Ethereum had the higher compund mean at 0.285405

Q. Which coin had the highest positive score?

A: Ethereum had the highest positive score.

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [32]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
stopwords_list = stopwords.words('english')

# Expand the default stopwords list if necessary
stop_words_addon = {'https', 
                    'U.Today', 
                    'oct',
                    'exchange',
                    'utoday',
                    'reutersdado',
                    'october',
                    'following',
                    'user',
                    'according',
                    'appreciationfrom',
                    'exchangetraded',
                    'representation',
                    'httpsifttteavj',
                    'wednesday',
                    'currently',
                    'fund',
                    'char', 
                    'chars', 
                    'SuSINGAPORE', 
                    'ha', 
                    'might', 
                    'well', 
                    'via', 
                    'vs', 
                    'hi',
                    'co',
                    'seen',
                    'taken',
                    'day',
                    'new',
                    'us',
                    'week',
                    'illustration',
                    'year',
                    'may',
                    'one',
                    'high',
                    'first'}

In [33]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Create a list of the words
    tokenized_words=word_tokenize(text)

    # Convert the words to lowercase
    lowercase_words=[word.lower() for word in tokenized_words]
    
    # Remove the punctuation
    regex=re.compile("[^a-zA-Z]")
    regex_words=[regex.sub("", word) for word in lowercase_words]
    
    # Remove the stop words
    stop_it=set(stopwords.words('english'))
    remove_stopwords=[word.lower() for word in regex_words if word not in stop_it.union(stop_words_addon)]
    
    # Lemmatize Words into root words
    lemmatizer=WordNetLemmatizer()
    tokens=[lemmatizer.lemmatize(word) for word in remove_stopwords]
    
    return tokens

In [34]:
# Create a new tokens column for Bitcoin
bitcoin_tokens = [tokenizer(headline) for headline in bitcoin_df["text"]]
bitcoin_df["tokens"] = bitcoin_tokens

bitcoin_df.head()

,date,text,compound,positive,negative,neutral,tokens
0,2021-11-05,A similar hoax earlier this year tied Walmart ...,-0.2732,0.000,0.063,0.937,"[similar, hoax, earlier, tied, walmart, liteco..."
1,2021-10-10,"Specifically, why did someone make a massive p...",0.5461,0.121,0.000,0.879,"[specifically, , someone, make, massive, purch..."
2,2021-10-28,Theres a big new presence slurping up power fr...,0.3612,0.096,0.000,0.904,"[there, big, presence, slurping, power, grid, ..."
3,2021-10-26,"For all the talk of democratizing finance, the...",0.0000,0.000,0.000,1.000,"[talk, democratizing, finance, , vast, majorit..."
4,2021-10-21,Representation of cryptocurrency Bitcoin is pl...,0.0000,0.000,0.000,1.000,"[cryptocurrency, bitcoin, placed, pc, motherbo..."


In [35]:
# Create a new tokens column for Ethereum
ethereum_tokens = [tokenizer(headline) for headline in bitcoin_df["text"]]
ethereum_df["Tokens"] = bitcoin_tokens

ethereum_df.head()

,date,text,compound,positive,negative,neutral,Tokens
0,2021-10-16,A new cross-chain bridge is currently connecte...,0.0000,0.000,0.0,1.000,"[similar, hoax, earlier, tied, walmart, liteco..."
1,2021-10-14,Mark Cuban has some advice for people who are ...,0.0000,0.000,0.0,1.000,"[specifically, , someone, make, massive, purch..."
2,2021-11-05,Ethereum and bitcoin are the two biggest crypt...,0.4588,0.094,0.0,0.906,"[there, big, presence, slurping, power, grid, ..."
3,2021-11-01,Elon Musk\r\npicture alliance / Getty Images\r...,0.5267,0.093,0.0,0.907,"[talk, democratizing, finance, , vast, majorit..."
4,2021-11-01,Cryptocurrency and business continuity line im...,0.4588,0.097,0.0,0.903,"[cryptocurrency, bitcoin, placed, pc, motherbo..."


---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [16]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [17]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [20]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---